<a href="https://colab.research.google.com/github/AkshathaBolla/ANN/blob/main/Completed_Module_I_Practice_AgriTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module I: Practice Exercise - 'Agri-Tech' (Regularization Edition)

### **Context: Smart Farming Sensors**
You are processing data from 1,000 soil sensors in a smart greenhouse.
**Inputs:**
1.  **Soil Moisture (%)**
2.  **UV Index (0-15)**
3.  **Pesticide Residue (ppm)**

**Goals:**
* **Level 1 (Classification):** Predict if the plant is **'Infected' (1)** or **'Healthy' (0)**.
* **Level 2 (Regression):** Predict the **Expected Yield (kg)**.

---
**INSTRUCTIONS:**
This exercise focuses on **L2 Regularization** and **Manual Loss Math**.
* **Level 2:** You must use **L2 Regularization (Weight Decay)** in the optimizer to handle overfitting.
* **Level 3:** You must implement **Binary Cross Entropy** from scratch.

In [10]:
# CELL 1: DATA GENERATION (Run this first)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(2024)

# 1000 Sensors
N = 1000
# Features: Moisture(20-90), UV(0-12), Pesticide(0-5)
X = torch.rand(N, 3) * torch.tensor([70, 12, 5]) + torch.tensor([20, 0, 0])

# Target A: Infection Status (Classification)
# Infected (1) if Moisture > 80 OR (UV < 3 AND Pesticide < 1)
risk_score = (X[:, 0] / 80) + (3 / (X[:, 1] + 1))
y_health = (risk_score > 1.5).float().view(-1, 1)

# Target B: Crop Yield (Regression)
# Yield = Moisture * 0.5 + UV * 2 - Pesticide * 4
y_yield = (X[:, 0] * 0.5) + (X[:, 1] * 2) - (X[:, 2] * 4) + torch.randn(N)*2
y_yield = y_yield.view(-1, 1).float()

print(f"Agri-Tech Data Ready. X: {X.shape}, y_health: {y_health.shape}, y_yield: {y_yield.shape}")

Agri-Tech Data Ready. X: torch.Size([1000, 3]), y_health: torch.Size([1000, 1]), y_yield: torch.Size([1000, 1])


## **Level 1: The Disease Detector (Classification)**
**Objective:** Build a standard classification model.

**The Blueprint:**
1.  **Architecture:** `Linear(3, 16) -> ReLU -> Linear(16, 1) -> Sigmoid`.
2.  **Training:**
    * Loss: `BCELoss`
    * Optimizer: `Adam` (lr=0.01)
    * Epochs: 50
    * **Constraint:** Train on the full dataset `X` and `y_health` (No DataLoaders yet).

In [11]:
# LEVEL 1: WRITE YOUR CODE HERE

# 1. Define 'DiseaseModel'
class DiseaseModel(nn.Module):
    # TODO: Linear -> ReLU -> Linear -> Sigmoid
    def __init__(self):
        super(DiseaseModel,self).__init__()

        self.layer1=nn.Linear(3,16)
        self.relu=nn.ReLU()
        self.layer2=nn.Linear(16,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):
       x=self.layer1(x)
       x=self.relu(x)
       x=self.layer2(x)
       x=self.sigmoid(x)
       return x
model_cls = DiseaseModel()


# 2. Optimizer & Loss
criterion=nn.BCELoss() #Binary Cross Entropy
optimizer=optim.Adam(model_cls.parameters(),lr=0.01)

# 3. Training Loop (Simple)
for epoch in range(50):
  optimizer.zero_grad()
  outputs=model_cls(X)
  loss=criterion(outputs,y_health)
  loss.backward()
  optimizer.step()

In [12]:
# TEST LEVEL 1
try:
    assert isinstance(list(model_cls.children())[-1], nn.Sigmoid), "Output layer must be Sigmoid for binary classification"
    assert list(model_cls.children())[0].out_features == 16, "Hidden layer must have 16 neurons"
    print("✅ Level 1 Passed: Classification Architecture correct.")
except Exception as e:
    print(f"❌ Level 1 Fail: {e}")

✅ Level 1 Passed: Classification Architecture correct.


## **Level 2: The Yield Predictor (Regularization)**
**Objective:** Predict Yield using **L2 Regularization** (Weight Decay) to prevent overfitting.

**Step 2.1: Data Pipeline**
* Split `X` and `y_yield` into **Train (800)** and **Test (200)**.
* Create DataLoaders (Batch Size = 64).

**Step 2.2: The Regularized Optimizer**
* Model: `Linear(3, 64) -> ReLU -> Linear(64, 64) -> ReLU -> Linear(64, 1)`.
* **Task:** Initialize the Adam optimizer with `weight_decay=0.05`.
* *Concept:* This penalizes large weights, preventing the model from memorizing noise. It is an alternative to Dropout.

In [13]:
from logging import CRITICAL
# LEVEL 2: WRITE YOUR CODE HERE

# 1. Split & DataLoaders
X_train,X_test=X[:800],X[800:]
y_train,y_test=y_yield[:800],y_yield[800:]

train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64)

# 2. Define 'YieldModel' (Deep: 64 -> 64)
class YieldModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(3,64),
        nn.ReLU(),
        nn.Linear(64,64),
        nn.ReLU(),
        nn.Linear(64,1)
    )
  def forward(self,x):
    return self.net(x)
model_reg = YieldModel()

# 3. Define Optimizer WITH Weight Decay
optimizer = optim.Adam(model_reg.parameters(), lr=0.01, weight_decay=0.05)
criterion=nn.MSELoss()

# 4. Training Loop
for epoch in range(50):
  for x_batch,y_batch in train_loader:
    optimizer.zero_grad()
    preds=model_reg(x_batch)
    loss=criterion(preds,y_batch)
    loss.backward()
    optimizer.step()

In [14]:
# TEST LEVEL 2
try:
    # Check for weight_decay
    if 'optimizer' in locals():
        wd = optimizer.defaults['weight_decay']
        assert wd == 0.05, f"Optimizer must have weight_decay=0.05, found {wd}"
        print("✅ Level 2 Passed: L2 Regularization configured.")
    else:
        print("⚠️ Level 2 Warning: 'optimizer' variable not found, but code might be correct.")
except Exception as e:
    print(f"❌ Level 2 Fail: {e}")

✅ Level 2 Passed: L2 Regularization configured.


## **Level 3: The Lab (Manual BCE)**

**Part 3.1: Manual Binary Cross Entropy**
Implement `my_bce_loss(pred, target)`.
$$ Loss = - (target * \log(pred) + (1 - target) * \log(1 - pred)) $$
*Hint: Add a tiny epsilon (1e-8) inside the log to avoid log(0) errors.*

**Part 3.2: The Tournament**
Compare these setups on the **Classification Task**:
1.  **"Standard"**: `ReLU` + `BCELoss`
2.  **"Manual"**: `LeakyReLU` + **YOUR** `my_bce_loss`
3.  **"Experimental"**: `GELU` + `BCELoss`

In [15]:
# LEVEL 3.1: MANUAL BCE LOSS
def my_bce_loss(pred, target):
    epsilon = 1e-8
    loss = - (target * torch.log(pred + epsilon) + (1-target) * torch.log(1 - pred + epsilon))
    return torch.mean(loss)

# LEVEL 3.2: CONFIG
experiments = [
    {"name": "Standard",     "act": nn.ReLU(), "loss": nn.BCELoss()}, # ReLU, BCELoss
    {"name": "Manual",       "act": nn.LeakyReLU(), "loss": my_bce_loss}, # LeakyReLU, my_bce_loss
    {"name": "Experimental", "act": nn.GELU(), "loss": nn.BCELoss}  # GELU, BCELoss
]

In [16]:
# TEST LEVEL 3
try:
    # Test Manual BCE
    p = torch.tensor([0.9]); t = torch.tensor([1.0])
    # Loss = -1 * log(0.9) approx 0.105
    val = my_bce_loss(p, t).item()
    assert abs(val - 0.105) < 0.01, f"Math Fail: Expected ~0.105, got {val}"

    assert isinstance(experiments[2]['act'], nn.GELU), "Exp 3 must be GELU"
    print("✅ Level 3 Passed: Manual BCE and Config correct.")
except Exception as e:
    print(f"❌ Level 3 Fail: {e}")

✅ Level 3 Passed: Manual BCE and Config correct.


## **Level 4: The Mechanic (Simple Gradient)**
**Objective:** Manual Gradient Descent on a single weight.

**Goal:** Learn $y = w \cdot x$

1.  **Data:** $x=4.0$, Target $y=12.0$
2.  **Weight:** $w=2.0$ (Initial guess)
3.  **Loss:** $L = (pred - y)^2$
4.  **Task:** Calculate gradient manually and update $w$.
    * $\frac{dL}{dw} = 2(pred - y) \cdot x$

In [17]:
# LEVEL 4: WRITE YOUR CODE HERE
lr = 0.01
x = 4.0
y = 12.0
w = 2.0

# 1. Forward (Pred)
pred = w*x
# 2. Gradient (dw)
grad = 2*(pred-y)*x

# 3. Update
w_new = w-lr*grad

In [18]:
# TEST LEVEL 4
try:
    # Pred = 2*4 = 8
    # Error = 8 - 12 = -4
    # Grad = 2 * -4 * 4 = -32
    # w_new = 2.0 - (0.01 * -32) = 2.0 + 0.32 = 2.32

    assert abs(w_new - 2.32) < 0.001, f"Math Fail: Expected 2.32, got {w_new}"
    print("✅ Level 4 Passed: Simple Gradient Descent successful!")
except Exception as e:
    print(f"❌ Level 4 Fail: {e}")

✅ Level 4 Passed: Simple Gradient Descent successful!
